In [15]:
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb


from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler



from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score


In [2]:
#arquivos

#caminho para rodar google colab
#arq_treinamento = pd.read_csv('treino.csv')
#arq_teste = pd.read_csv('teste.csv')


#caminhos para rodar localmente
arq_treinamento = pd.read_csv('arquivos/brutos/treino.csv')
arq_teste = pd.read_csv('arquivos/brutos/teste.csv')


def arq_treinamento_info():
    
    print(arq_treinamento.head())
    print(arq_treinamento.info())
    print(arq_treinamento.describe())
    print(arq_treinamento.isnull().sum())


#definindo target e características arquivo de treinamento
arq_treinamento_x = arq_treinamento.drop(columns=['target', 'id'])

arq_treinamento_y = arq_treinamento['target']



#train_test_split dos dados de teste

X_train, X_test, y_train, y_test = train_test_split(arq_treinamento_x, arq_treinamento_y, test_size=0.2, random_state=42)

#targets estao desbalanceados
#arq_treinamento_y.value_counts()
y_train.value_counts()

def split_info():
    print("Dados de treinamento (X_train):")
    print(X_train)
    print("Labels de treinamento (y_train):")
    print(y_train)
    print("Dados de teste (X_test):")
    print(X_test)
    print("Labels de teste (y_test):")
    print(y_test)



In [4]:
#treinamento do RandomForest, XGB e NeuralNerwork com dados balanceados com SMOTETomek ADASYN
#usando Pipeline #validacao_cruzada #f1_score

random_state_42 = 42

balanceador = Pipeline([
    ('adasyn', ADASYN(random_state=random_state_42)),
    ('smote_tomek_balanceamento', SMOTETomek(random_state=random_state_42))
])


modelos = {
    "Random Forest": RandomForestClassifier(random_state=random_state_42),
    "XGBoost": xgb.XGBClassifier(objective="multi:softmax", num_class=5, random_state=random_state_42),
    "Neural Network": MLPClassifier(max_iter=1000)
}

#validacao cruzada com stratifieldkfold
validacao_cruzada = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state_42)

resultados = []

for name, modelo in modelos.items():
    f1_macro_scores = []
    f1_micro_scores = []
    f1_balanceada_scores = []
    accuracy_scores = []

    for treino_dados, validacao_dados in validacao_cruzada.split(X_train, y_train):
        X_train_vc, X_val_vc = X_train.iloc[treino_dados], X_train.iloc[validacao_dados]
        y_train_vc, y_val_vc = y_train.iloc[treino_dados], y_train.iloc[validacao_dados]


        X_rebalanceado, y_rebalanceado = balanceador.fit_resample(X_train_vc, y_train_vc)

        modelo.fit(X_rebalanceado, y_rebalanceado)

        y_pred = modelo.predict(X_val_vc)

        ac_score = accuracy_score(y_val_vc, y_pred) #nao será muito útil por que os dados são multiclasse, mas pode ser interessante verificar diferenças gritantes
        f1_balanceada = f1_score(y_val_vc, y_pred, average='weighted')
        f1_macro = f1_score(y_val_vc, y_pred, average='macro')
        f1_micro = f1_score(y_val_vc, y_pred, average='micro')

        accuracy_scores.append(ac_score)
        f1_macro_scores.append(f1_macro)
        f1_micro_scores.append(f1_micro)
        f1_balanceada_scores.append(f1_balanceada)


        """matriz_confusao = confusion_matrix(y_val_vc, y_pred)
        display = ConfusionMatrixDisplay(confusion_matrix=matriz_confusao)
        display.plot(cmap='Blues')
        plt.title(f'{name} - Fold')
        print(name, matriz_confusao)
        plt.show()"""

    #resultados: média das métricas dos folds

    """resultados.append({
        "Model": name,
        "Accuracy": np.mean(accuracy_scores),
        "F1 Macro": np.mean(f1_macro_scores),
        "F1 Micro": np.mean(f1_micro_scores),
        "F1 Balanceada": np.mean(f1_balanceada_scores)
    })"""

    # priorizando a métrica f1-score que será cobrada
    resultados.append({
        "Model": name,
      # "Accuracy": np.mean(accuracy_scores),
       # "F1 Macro": np.mean(f1_macro_scores),
       # "F1 Micro": np.mean(f1_micro_scores),
        "F1 Balanceada": np.mean(f1_balanceada_scores)
    })


print(pd.DataFrame(resultados))



            Model  F1 Balanceada
0   Random Forest       0.757623
1         XGBoost       0.748710
2  Neural Network       0.704308


In [5]:
#Neural Network não deu resultado
#GridSearchCV não foi possível por conta do Hardware. 

In [9]:
#Testando RandomSearchCV com RandomForest
#fazendo balanceamento dos dados fora da validacao cruzada para fazer o randomsearchCV

random_state_42 = 42

balanceador = Pipeline([
    ('adasyn', ADASYN(random_state=random_state_42)),
    ('smote_tomek_balanceamento', SMOTETomek(random_state=random_state_42))
])

X_rebalanceado_smote_tomek_adasyn, y_rebalanceado_smote_tomek_adasyn = balanceador.fit_resample(
    X_train, y_train)

#print(X_rebalanceado_smote_tomek_adasyn)
#print(y_rebalanceado_smote_tomek_adasyn)

In [12]:
#Testando RandomSearchCV com RandomForest

parametros_dicionario = {
    'n_estimators': [100, 300, 500, 800], 
    'max_depth': [None, 10, 20, 30], 
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 4], 
    'max_features': ['sqrt', 'log2']
}


random_modelo = RandomForestClassifier(random_state=42)


random_search_config = RandomizedSearchCV(
    random_modelo, parametros_dicionario, 
    n_iter=10,  #menos testes para conseguir rodar
    scoring='f1_weighted', 
    cv=3,  # Validação cruzada com 3 folds (mais rápido)
    verbose=2, 
    n_jobs=-1,  # Usa todos os núcleos do processador
    random_state=42
)

random_search_config.fit(X_rebalanceado_smote_tomek_adasyn, y_rebalanceado_smote_tomek_adasyn)

print("melhores parametros:", random_search_config.best_params_)

#resultados:
#melhores parametros: {'n_estimators': 300, 'min_samples_split': 2, 
#'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 20}


Fitting 3 folds for each of 10 candidates, totalling 30 fits
melhores parametros: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 20}


In [ ]:
#Testando RandomSearchCV com XGB

parametros_dicionario = {
    'n_estimators': [100, 300, 500, 800], 
    'max_depth': [3, 5, 10, 20], 
    'learning_rate': [0.01, 0.05, 0.1, 0.2], 
    'subsample': [0.6, 0.8, 1.0], 
    'colsample_bytree': [0.6, 0.8, 1.0]
}

xgb_modelo = XGBClassifier(random_state=42, eval_metric="mlogloss")

random_search_xgb = RandomizedSearchCV(
    xgb_modelo, parametros_dicionario, 
    n_iter=10,  # menos testes
    scoring='f1_weighted', 
    cv=3,  # Validação cruzada com 3 folds (mais rápido)
    verbose=2, 
    n_jobs=-1,  # Usa todos os núcleos do processador
    random_state=42
)

random_search_xgb.fit(X_rebalanceado_smote_tomek_adasyn, y_rebalanceado_smote_tomek_adasyn)

# Exibir melhores parâmetros
print("Melhores parâmetros", random_search_xgb.best_params_)

#resultados:
#Melhores parâmetros {'subsample': 1.0, 'n_estimators': 500, 
#'max_depth': 20, 'learning_rate': 0.2, 'colsample_bytree': 0.8}



Fitting 3 folds for each of 10 candidates, totalling 30 fits
